<a href="https://colab.research.google.com/github/cccg8105/notebook_AD/blob/master/Estadistica%20no%20parametrica/AnalisisDistribuciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analisis de distribuciones